## Creating confidence intervals in python
In this assessment, you will look at data from a study on toddler sleep habits. 

The confidence intervals you create and the questions you answer in this Jupyter notebook will be used to answer questions in the following graded assignment.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import t
pd.set_option('display.max_columns', 30) # set so can see all columns of the DataFrame

Your goal is to analyse data which is the result of a study that examined
differences in a number of sleep variables between napping and non-napping toddlers. Some of these
sleep variables included: Bedtime (lights-off time in decimalized time), Night Sleep Onset Time (in
decimalized time), Wake Time (sleep end time in decimalized time), Night Sleep Duration (interval
between sleep onset and sleep end in minutes), and Total 24-Hour Sleep Duration (in minutes). Note:
[Decimalized time](https://en.wikipedia.org/wiki/Decimal_time) is the representation of the time of day using units which are decimally related.   


The 20 study participants were healthy, normally developing toddlers with no sleep or behavioral
problems. These children were categorized as napping or non-napping based upon parental report of
children’s habitual sleep patterns. Researchers then verified napping status with data from actigraphy (a
non-invasive method of monitoring human rest/activity cycles by wearing of a sensor on the wrist) and
sleep diaries during the 5 days before the study assessments were made.


You are specifically interested in the results for the Bedtime, Night Sleep Duration, and Total 24-
Hour Sleep Duration. 

Reference: Akacem LD, Simpkin CT, Carskadon MA, Wright KP Jr, Jenni OG, Achermann P, et al. (2015) The Timing of the Circadian Clock and Sleep Differ between Napping and Non-Napping Toddlers. PLoS ONE 10(4): e0125181. https://doi.org/10.1371/journal.pone.0125181

In [2]:
# Import the data
df = pd.read_csv("nap_no_nap.csv") 

In [6]:
# First, look at the DataFrame to get a sense of the data
df


,id,sex,age (months),dlmo time,days napped,napping,nap lights outl time,nap sleep onset,nap midsleep,nap sleep offset,nap wake time,nap duration,nap time in bed,night bedtime,night sleep onset,sleep onset latency,night midsleep time,night wake time,night sleep duration,night time in bed,24 h sleep duration,bedtime phase difference,sleep onset phase difference,midsleep phase difference,wake time phase difference
0,1,female,33.7,19.24,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1245.0,20.68,0.23,1.92,7.17,629.40,643.00,629.40,-1.21,-1.44,6.68,11.93
1,2,female,31.5,18.27,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1163.0,19.48,0.25,1.09,6.69,672.40,700.40,672.40,-0.96,-1.21,6.82,12.42
2,3,male,31.9,19.14,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1200.0,20.05,0.45,1.29,6.53,628.80,682.60,628.80,-0.46,-0.91,6.15,11.39
3,4,female,31.6,19.69,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1186.0,19.50,0.05,1.89,8.28,766.60,784.00,766.60,0.23,0.19,6.20,12.59
4,5,female,33.0,19.52,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1161.0,19.65,0.45,1.30,6.95,678.00,718.00,678.00,0.31,-0.13,5.78,11.43
5,6,female,36.2,18.22,4,1,14.00,14.22,15.00,15.78,16.28,93.75,137.00,1235.0,20.25,0.29,1.26,6.28,602.20,653.80,695.95,-1.73,-2.03,7.05,12.06
6,7,male,36.3,19.28,1,1,14.75,15.03,15.92,16.80,16.08,106.00,80.00,1260.0,20.96,0.36,2.12,7.27,618.40,655.40,724.40,-1.32,-1.68,6.84,11.99
7,8,male,30.0,21.06,5,1,13.09,13.43,14.44,15.46,15.82,121.60,163.80,1321.0,22.53,0.51,2.92,7.31,526.80,582.40,648.40,-0.95,-1.47,5.86,10.25
8,9,male,33.2,19.38,2,1,14.41,14.42,15.71,17.01,16.60,155.50,131.25,1224.0,20.37,0.13,1.60,6.82,626.80,660.33,782.30,-0.86,-0.99,6.22,11.44
9,10,female,37.1,19.93,3,1,13.12,13.42,14.31,15.19,15.30,106.67,130.67,1278.0,21.63,0.84,2.20,6.52,549.50,626.00,656.17,-0.76,-1.82,6.21,10.59


**Question**: What variable is used in the column 'napping' to indicate a toddler takes a nap?


**Question**: What is the sample size $n$?

## Hypothesis testing
We will look at two hypothesis test, each with $\alpha = .025$:  


1. Is the average bedtime for toddlers who nap later than the average bedtime for toddlers who don't nap?


$$H_0: \mu_{nap}=\mu_{no\ nap}, \ H_a:\mu_{nap}>\mu_{no\ nap}$$
Or equivalently:
$$H_0: \mu_{nap}-\mu_{no\ nap}=0, \ H_a:\mu_{nap}-\mu_{no\ nap}>0$$


2. The average 24 h sleep duration (in minutes) for napping toddlers is different from toddlers who don't nap.


$$H_0: \mu_{nap}=\mu_{no\ nap}, \ H_a:\mu_{nap}\neq\mu_{no\ nap}$$
Or equivalenty:
$$H_0: \mu_{nap}-\mu_{no\ nap}=0, \ H_a:\mu_{nap}-\mu_{no\ nap} \neq 0$$

Aside: This $\alpha$ level is equivalent to $\alpha = .05$ and then applying the [Bonferonni correction](https://en.wikipedia.org/wiki/Bonferroni_correction).

Before any analysis, we will convert 'night bedtime' into decimalized time. 

In [5]:
# Convert 'night bedtime' into decimalized time
df.loc[:,'night bedtime'] = np.floor(df['night bedtime'])*60 + np.round(df['night bedtime']%1,2 )*100

Now, isolate the column 'night bedtime' for those who nap into a new variable, and those who didn't nap into another new variable. 

In [9]:
nap=df.query('napping == 0')
nap

,id,sex,age (months),dlmo time,days napped,napping,nap lights outl time,nap sleep onset,nap midsleep,nap sleep offset,nap wake time,nap duration,nap time in bed,night bedtime,night sleep onset,sleep onset latency,night midsleep time,night wake time,night sleep duration,night time in bed,24 h sleep duration,bedtime phase difference,sleep onset phase difference,midsleep phase difference,wake time phase difference
0,1,female,33.7,19.24,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1245.0,20.68,0.23,1.92,7.17,629.4,643.0,629.4,-1.21,-1.44,6.68,11.93
1,2,female,31.5,18.27,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1163.0,19.48,0.25,1.09,6.69,672.4,700.4,672.4,-0.96,-1.21,6.82,12.42
2,3,male,31.9,19.14,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1200.0,20.05,0.45,1.29,6.53,628.8,682.6,628.8,-0.46,-0.91,6.15,11.39
3,4,female,31.6,19.69,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1186.0,19.50,0.05,1.89,8.28,766.6,784.0,766.6,0.23,0.19,6.20,12.59
4,5,female,33.0,19.52,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1161.0,19.65,0.45,1.30,6.95,678.0,718.0,678.0,0.31,-0.13,5.78,11.43


In [10]:
no_nap=df.query('napping == 1')
no_nap

,id,sex,age (months),dlmo time,days napped,napping,nap lights outl time,nap sleep onset,nap midsleep,nap sleep offset,nap wake time,nap duration,nap time in bed,night bedtime,night sleep onset,sleep onset latency,night midsleep time,night wake time,night sleep duration,night time in bed,24 h sleep duration,bedtime phase difference,sleep onset phase difference,midsleep phase difference,wake time phase difference
5,6,female,36.2,18.22,4,1,14.00,14.22,15.00,15.78,16.28,93.75,137.00,1235.0,20.25,0.29,1.26,6.28,602.20,653.80,695.95,-1.73,-2.03,7.05,12.06
6,7,male,36.3,19.28,1,1,14.75,15.03,15.92,16.80,16.08,106.00,80.00,1260.0,20.96,0.36,2.12,7.27,618.40,655.40,724.40,-1.32,-1.68,6.84,11.99
7,8,male,30.0,21.06,5,1,13.09,13.43,14.44,15.46,15.82,121.60,163.80,1321.0,22.53,0.51,2.92,7.31,526.80,582.40,648.40,-0.95,-1.47,5.86,10.25
8,9,male,33.2,19.38,2,1,14.41,14.42,15.71,17.01,16.60,155.50,131.25,1224.0,20.37,0.13,1.60,6.82,626.80,660.33,782.30,-0.86,-0.99,6.22,11.44
9,10,female,37.1,19.93,3,1,13.12,13.42,14.31,15.19,15.30,106.67,130.67,1278.0,21.63,0.84,2.20,6.52,549.50,626.00,656.17,-0.76,-1.82,6.21,10.59
10,11,male,32.9,18.79,4,1,13.99,14.03,14.85,15.68,16.10,98.75,126.60,1185.0,19.88,0.44,1.34,6.80,655.20,694.80,753.95,-0.66,-1.09,6.55,12.01
11,12,female,35.0,19.65,5,1,13.18,13.45,14.33,15.21,15.35,105.80,130.40,1218.0,20.84,0.66,1.93,7.03,611.20,660.40,717.00,-0.53,-1.19,6.28,11.38
12,13,male,35.1,19.83,3,1,13.94,14.48,15.26,16.03,15.78,93.33,110.20,1222.0,20.89,0.67,1.99,7.09,611.80,662.20,705.13,-0.39,-1.06,6.16,11.26
13,14,female,35.6,19.88,4,1,12.68,13.08,13.92,14.76,15.00,100.75,139.33,1226.0,20.80,0.54,1.96,7.11,618.80,671.20,719.55,-0.38,-0.92,6.08,11.23
14,15,female,36.6,19.94,4,1,12.71,12.88,13.80,14.72,14.88,110.75,130.00,1228.0,20.92,0.64,1.49,6.33,548.00,595.00,658.75,-0.34,-0.90,5.64,10.39


In [11]:
nap_bedtime = nap['night bedtime']
nap_bedtime

0    1245.0
1    1163.0
2    1200.0
3    1186.0
4    1161.0
Name: night bedtime, dtype: float64

In [12]:
no_nap_bedtime = no_nap['night bedtime']
no_nap_bedtime

5     1235.0
6     1260.0
7     1321.0
8     1224.0
9     1278.0
10    1185.0
11    1218.0
12    1222.0
13    1226.0
14    1228.0
15    1246.0
16    1243.0
17    1202.0
18    1190.0
19    1218.0
Name: night bedtime, dtype: float64

Now find the sample mean bedtime for nap and no_nap.

In [13]:
nap_mean_bedtime = nap_bedtime.mean()
nap_mean_bedtime

1191.0

In [14]:
no_nap_mean_bedtime = no_nap_bedtime.mean()
no_nap_mean_bedtime

1233.0666666666666

In [16]:
mean_diff=nap_mean_bedtime - no_nap_mean_bedtime
mean_diff

-42.066666666666606

**Question**: What is the sample difference of mean bedtime for nappers minus no nappers?

Now find the sample standard deviation for $X_{nap}$ and $X_{no\ nap}$.

In [30]:

nap_s_bedtime = np.std(nap_bedtime)
nap_s_bedtime

30.678983033992505

In [31]:
no_nap_s_bedtime = np.std(no_nap_bedtime)
no_nap_s_bedtime

33.277553328866524

**Question**: What is the s.e.$(\bar{X}_{nap} - \bar{X}_{no\ nap})$?

We expect the variance in sleep time for toddlers who nap and toddlers who don't nap to be the same. So we use a pooled standard error.

Calculate the pooled standard error of $\bar{X}_{nap} - \bar{X}_{no\ nap}$ using the formula below.

$s.e.(\bar{X}_{nap} - \bar{X}_{no\ nap}) = \sqrt{\frac{(n_1-1)s_1^2 + (n_2-1)s_2^2}{n_1+n_2-2}(\frac{1}{n_1}+\frac{1}{n_2})}$

In [32]:
n1=len(nap_bedtime)
n2=len(no_nap_bedtime)
(n1 , n2)

(5, 15)

In [33]:
num1=(n1-1)*(nap_s_bedtime*nap_s_bedtime) + (n2-1)*(no_nap_s_bedtime*no_nap_s_bedtime)
den1=n1+n2-2
num2= (1/n1 + 1/n2)
pooled_se = np.sqrt((num1 * num2)/ den1)
pooled_se

16.895468503923713

**Question**: Given our sample size of $n$, how many degrees of freedom ($df$) are there for the associated $t$ distribution?

Now calculate the $t$-test statistic for our first hypothesis test using  
* pooled s.e.($\bar{X}_{nap} - \bar{X}_{no\ nap}$)  
* $\bar{X}_{nap} - \bar{X}_{no\ nap}$  
* $\mu_{0,\ nap} - \mu_{0,\ no\ nap}=0$, the population difference in means under the null hypothesis

In [36]:
t_statistic=(nap_mean_bedtime-no_nap_mean_bedtime)/pooled_se
t_statistic

-2.4898194836619814

**Question**: What is the p-value for the first hypothesis test?

To find the p-value, we can use the function:
```
t.cdf(y, df)
```
Which for $X \sim t(df)$ returns $P(X \leq y)$.

Because of the symmetry of the $t$ distrubution, we have that 
```
1-t.cdf(y, df)
```
returns $P(X > y)$

The function t.cdf(y, df) will give you the same value as finding the one-tailed probability of y on a t-table with the specified degrees of freedom.

Use the function t.cdf(y, df) to find the p-value for the first hypothesis test.

In [39]:
deg_fre=18
p_val=1-t.cdf(np.abs(t_statistic), deg_fre)
p_val

0.011392413734599205

**Question**: What are the t-statistic and p-value for the second hypothesis test?

Calculate the $t$ test statistics and corresponding p-value using the scipy function scipy.stats.ttest_ind(a, b, equal_var=True) and check with your answer. 

**Question**: Does scipy.stats.ttest_ind return values for a one-sided or two-sided test?

**Question**: Can you think of a way to recover the results you got using 1-t.cdf from the p-value given by scipy.stats.ttest_ind?

Use the scipy function scipy.stats.ttest_ind(a, b, equal_var=True) to find the $t$ test statistic and corresponding p-value for the second hypothesis test.

In [40]:
from scipy import stats

stats.ttest_ind(nap_bedtime, no_nap_bedtime, equal_var=True)

Ttest_indResult(statistic=-2.367160052079275, pvalue=0.029334902861805394)

In [26]:
nap_24=nap["24 h sleep duration"]
nap_24

0    629.4
1    672.4
2    628.8
3    766.6
4    678.0
Name: 24 h sleep duration, dtype: float64

In [27]:
no_nap24=no_nap["24 h sleep duration"]
no_nap24

5     695.95
6     724.40
7     648.40
8     782.30
9     656.17
10    753.95
11    717.00
12    705.13
13    719.55
14    658.75
15    710.58
16    724.00
17    663.40
18    766.40
19    707.00
Name: 24 h sleep duration, dtype: float64

In [28]:
import scipy
from scipy import stats
scipy.stats.ttest_ind(nap_24, no_nap24, equal_var=True) 

Ttest_indResult(statistic=-1.4811248223284985, pvalue=0.1558664953018476)

**Question**: For the $\alpha=.025$, do you reject or fail to reject the first hypothesis?

**Question**: For the $\alpha=.025$, do you reject or fail to reject the second hypothesis?